# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import SQLAlchemy's create_engine
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# Merge the messages and categories datasets using the common 'id'
df = pd.merge(messages, categories, on='id')

# Display the first few rows of the combined dataset
df.head()

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# Create a DataFrame of the 36 individual category columns
categories = df['categories'].str.split(';', expand=True)

# Display the first few rows of the categories DataFrame
categories.head()

In [ ]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: x[:-2])
print(category_colnames)

In [ ]:
# Rename the columns of `categories`
categories.columns = category_colnames

# Display the first few rows of the updated categories DataFrame
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories:
    # Set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # Convert column from string to numeric
    categories[column] = categories[column].astype(int)

# Display the first few rows of the updated categories DataFrame
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# Drop the original categories column from df
df = df.drop('categories', axis=1)
df.head()

In [ ]:
# Concatenate the original dataframe with the new categories dataframe
df = pd.concat([df, categories], axis=1)

# Display the first few rows of the updated dataframe
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# Check the number of duplicates in the dataset
num_duplicates = df.duplicated().sum()
print(f"Number of duplicates before removal: {num_duplicates}")

In [ ]:
# Drop duplicates
df = df.drop_duplicates()

In [ ]:
# Check the number of duplicates again to confirm removal
num_duplicates_after = df.duplicated().sum()
print(f"Number of duplicates after removal: {num_duplicates_after}")

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
# Create a database engine
engine = create_engine('sqlite:///DisasterResponse.db')

# Save the DataFrame to the SQLite database
df.to_sql('DisasterResponse', engine, index=False)

print("Data saved to SQLite database successfully.")

### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.

In [ ]:
# def load_data(messages_filepath, categories_filepath):
#     """
#     Load data from CSV files.
    
#     Parameters:
#     messages_filepath (str): Path to the messages dataset.
#     categories_filepath (str): Path to the categories dataset.
    
#     Returns:
#     df (DataFrame): Merged DataFrame.
#     """
#     # Load messages dataset
#     messages = pd.read_csv(messages_filepath)
    
#     # Load categories dataset
#     categories = pd.read_csv(categories_filepath)
    
#     # Merge datasets
#     df = pd.merge(messages, categories, on='id')
    
#     return df

# def clean_data(df):
#     """
#     Clean the merged DataFrame.
    
#     Parameters:
#     df (DataFrame): Merged DataFrame.
    
#     Returns:
#     df (DataFrame): Cleaned DataFrame.
#     """
#     # Split categories into separate category columns
#     categories = df['categories'].str.split(';', expand=True)
    
#     # Extract column names
#     row = categories.iloc[0]
#     category_colnames = row.apply(lambda x: x[:-2])
#     categories.columns = category_colnames
    
#     # Convert category values to 0 or 1
#     for column in categories:
#         categories[column] = categories[column].str[-1].astype(int)
    
#     # Replace categories column in df with new category columns
#     df = df.drop('categories', axis=1)
#     df = pd.concat([df, categories], axis=1)
    
#     # Remove duplicates
#     df = df.drop_duplicates()
    
#     return df

# def save_data(df, database_filename):
#     """
#     Save the cleaned DataFrame to an SQLite database.
    
#     Parameters:
#     df (DataFrame): Cleaned DataFrame.
#     database_filename (str): Path to the SQLite database.
#     """
#     engine = create_engine(f'sqlite:///{database_filename}')
#     df.to_sql('DisasterResponse', engine, index=False, if_exists='replace')
#     print("Data saved to SQLite database successfully.")

# # Simulate command-line arguments
# messages_filepath = 'messages.csv'
# categories_filepath = 'categories.csv'
# database_filepath = 'DisasterResponse.db'

# print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'.format(messages_filepath, categories_filepath))
# df = load_data(messages_filepath, categories_filepath)
# print('Cleaning data...')
# df = clean_data(df)
# print('Saving data...\n    DATABASE: {}'.format(database_filepath))
# save_data(df, database_filepath)
# print('Data processing completed.')